In [1]:
from utils import *
from dataio import *
from faraday_stuff import *
from qufit import *
from qu_models import *

In [2]:
cfg_file = 'configs/xmmlss12.cfg'
cfg = QUcfg()              # init config class
cfg.read_cfg(cfg_file)     # read config file

In [3]:
qufit = QUfit(cfg)
qufit.run_file()

------
Fitting ID2096
Model SFT
------


AttributeError: 'QUSimple' object has no attribute 'l2'